# Compare Waveframe V4.0 to ΛCDM

This notebook wires together the Equations modules to:

1. Define an α(z) profile (constant / logistic / power-law in a).
2. Solve for H(z) using `hubble_entropy_growth.py`.
3. Build a reference ΛCDM model from `limit_case_LCDM.py`.
4. Plot H(z) and residuals H/H_LCDM − 1.

**Note:** The notebook assumes the `Equations` folder is two levels up from this notebook.


In [ ]:
# Setup: imports and sys.path so we can import from Equations
import sys, os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add the Equations directory to the path (adjust if your layout differs)
root = Path.cwd().resolve()
eq_dir_candidates = [
    root.parents[1] / 'Equations',   # expected when this notebook lives in analysis/notebooks
    root.parents[0] / 'Equations',   # fallback
]
for cand in eq_dir_candidates:
    if cand.exists():
        sys.path.insert(0, str(cand))
        break

from hubble_entropy_growth import (
    make_alpha, solve_H_of_z, infer_alpha_match_from_H
)
from limit_case_LCDM import H_LCDM_flat

print('Imports ok. Using Equations at:', cand)

In [ ]:
# Define redshift grid and baseline cosmology
z = np.linspace(0.0, 2.0, 400)
H0 = 70.0  # km/s/Mpc (units arbitrary as long as consistent)
Omega_m0 = 0.3
Omega_r0 = 0.0
Omega_L0 = 1.0 - Omega_m0 - Omega_r0

# Reference ΛCDM H(z)
H_LCDM = H_LCDM_flat(H0, Omega_m0, Omega_r0, Omega_L0)
H_ref = H_LCDM(z)
print('Reference H(z) computed. H(0)=', H_ref[0])

In [ ]:
# Choose an α(z) profile for Waveframe
# Options: constant, powerlaw_a, logistic
alpha_spec = { 'type': 'logistic', 'zc': 0.7, 'width': 0.25, 'low': 0.0, 'high': 0.02 }
alpha = make_alpha(alpha_spec)

# Solve Waveframe H(z)
H_wf = solve_H_of_z(z, H0, alpha)
print('Waveframe H(0)=', H_wf[0])

In [ ]:
# Plot H(z) comparison
plt.figure()
plt.plot(z, H_ref, label='ΛCDM H(z)')
plt.plot(z, H_wf, label='Waveframe H(z)')
plt.xlabel('z')
plt.ylabel('H(z)')
plt.legend()
plt.title('H(z): Waveframe vs ΛCDM')
plt.show()

# Plot residuals: H/H_LCDM − 1
resid = H_wf / H_ref - 1.0
plt.figure()
plt.plot(z, resid, label='Residual')
plt.axhline(0, linestyle='--')
plt.xlabel('z')
plt.ylabel('H/H_LCDM − 1')
plt.title('Residuals')
plt.show()

In [ ]:
# Optional: exact matching demo — infer α_match from H_LCDM and re-solve to verify
alpha_match = infer_alpha_match_from_H(H_LCDM, z)
H_match = solve_H_of_z(z, H0, (lambda zz: np.interp(zz, z, alpha_match)))

plt.figure()
plt.plot(z, H_ref, label='ΛCDM H(z)')
plt.plot(z, H_match, label='Reconstructed via α_match', linestyle=':')
plt.xlabel('z')
plt.ylabel('H(z)')
plt.legend()
plt.title('Verification: α_match reproduces ΛCDM')
plt.show()